# Preparation for Colab

Make sure you're running a GPU runtime; if not, select "GPU" as the hardware accelerator in Runtime > Change Runtime Type in the menu. The next cells will print the CUDA version of the runtime if it has a GPU, and install PyTorch 1.7.1.

The following command installs the `clip` module from its source:

In [183]:
import numpy as np
import torch
import clip
from tqdm.notebook import tqdm
import pandas as pd

print("Torch version:", torch.__version__)

Torch version: 1.7.1+cu110


# Loading the model

Download and instantiate a CLIP model using the `clip` module that we just installed.

In [65]:
clip.available_models()

['RN50', 'RN101', 'RN50x4', 'ViT-B/32']

In [66]:
model, preprocess = clip.load('ViT-B/32')

In [67]:
input_resolution = model.input_resolution.item()
context_length = model.context_length.item()
vocab_size = model.vocab_size.item()

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

Model parameters: 151,277,313
Input resolution: 224
Context length: 77
Vocab size: 49408


In [ ]:
input_resolution = model.input_resolution.item()
context_length = model.context_length.item()
vocab_size = model.vocab_size.item()

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

# Preparing ImageNet labels and prompts

The following cell contains the 1,000 labels for the ImageNet dataset, followed by the text templates we'll use as "prompt engineering".

In [94]:
model.transformer
model.token_embedding
model.ln_final

RecursiveScriptModule(
  original_name=Multimodal
  (visual): RecursiveScriptModule(
    original_name=VisualTransformer
    (conv1): RecursiveScriptModule(original_name=Conv2d)
    (ln_pre): RecursiveScriptModule(original_name=LayerNorm)
    (transformer): RecursiveScriptModule(
      original_name=Transformer
      (resblocks): RecursiveScriptModule(
        original_name=Sequential
        (0): RecursiveScriptModule(
          original_name=ResidualAttentionBlock
          (attn): RecursiveScriptModule(
            original_name=MultiheadAttention
            (out_proj): RecursiveScriptModule(original_name=_LinearWithBias)
          )
          (ln_1): RecursiveScriptModule(original_name=LayerNorm)
          (mlp): RecursiveScriptModule(
            original_name=Sequential
            (c_fc): RecursiveScriptModule(original_name=Linear)
            (gelu): RecursiveScriptModule(original_name=QuickGELU)
            (c_proj): RecursiveScriptModule(original_name=Linear)
          )
   

In [68]:
torch.set_printoptions(precision=30)
for p in model.visual.parameters():
    print(p)

tensor([ 2.059104619547724723815917968750e-03,
         7.910715066827833652496337890625e-04,
        -7.474707905203104019165039062500e-03,
        -7.663656491786241531372070312500e-03,
        -6.317060440778732299804687500000e-03,
        -2.669303677976131439208984375000e-03,
         1.299867313355207443237304687500e-02,
        -5.165863782167434692382812500000e-03,
         6.902253031730651855468750000000e-01,
        -7.297636475414037704467773437500e-03,
        -1.365523487329483032226562500000e-01,
        -5.928675644099712371826171875000e-03,
         5.697516258805990219116210937500e-03,
        -2.657639235258102416992187500000e-02,
        -4.348624497652053833007812500000e-03,
        -4.882962908595800399780273437500e-03,
         2.282689325511455535888671875000e-02,
        -4.271556157618761062622070312500e-03,
        -1.928843557834625244140625000000e-02,
        -4.085517430212348699569702148438e-04,
         1.005410030484199523925781250000e-02,
        -1.14

tensor([-0.375244140625000000000000000000, -0.659667968750000000000000000000,
        -0.191894531250000000000000000000,  ...,
        -0.467041015625000000000000000000, -0.464599609375000000000000000000,
        -0.145874023437500000000000000000], device='cuda:0',
       dtype=torch.float16, requires_grad=True)
tensor([[ 0.021453857421875000000000000000,  0.003784179687500000000000000000,
         -0.010612487792968750000000000000,  ...,
         -0.004341125488281250000000000000,  0.005256652832031250000000000000,
         -0.009422302246093750000000000000],
        [ 0.009071350097656250000000000000,  0.003351211547851562500000000000,
         -0.017318725585937500000000000000,  ...,
          0.011917114257812500000000000000,  0.004051208496093750000000000000,
         -0.012260437011718750000000000000],
        [ 0.000643253326416015625000000000,  0.011009216308593750000000000000,
          0.001266479492187500000000000000,  ...,
          0.001676559448242187500000000000, -0.0061

tensor([ 5.270385742187500000000000000000e-02,
        -8.581542968750000000000000000000e-02,
         6.787109375000000000000000000000e-02,
         1.378631591796875000000000000000e-02,
        -1.483154296875000000000000000000e-02,
        -2.116394042968750000000000000000e-02,
         3.929138183593750000000000000000e-04,
         6.652832031250000000000000000000e-02,
        -8.966064453125000000000000000000e-02,
        -9.323120117187500000000000000000e-03,
        -2.995300292968750000000000000000e-02,
        -2.267837524414062500000000000000e-03,
         4.586791992187500000000000000000e-02,
        -1.027679443359375000000000000000e-02,
         2.767562866210937500000000000000e-03,
        -5.230712890625000000000000000000e-02,
         1.884765625000000000000000000000e-01,
        -3.796386718750000000000000000000e-02,
         6.427001953125000000000000000000e-02,
         3.108215332031250000000000000000e-02,
        -3.057861328125000000000000000000e-02,
        -2.66

tensor([ 3.579711914062500000000000000000e-02,
        -4.458618164062500000000000000000e-02,
         2.458190917968750000000000000000e-02,
        -3.013610839843750000000000000000e-02,
        -1.309204101562500000000000000000e-02,
        -1.026153564453125000000000000000e-02,
        -5.264282226562500000000000000000e-03,
        -7.621765136718750000000000000000e-03,
        -1.722717285156250000000000000000e-02,
        -2.281188964843750000000000000000e-02,
         3.716945648193359375000000000000e-04,
         4.577636718750000000000000000000e-02,
        -1.530456542968750000000000000000e-02,
         1.319122314453125000000000000000e-02,
         3.192138671875000000000000000000e-02,
         7.836914062500000000000000000000e-02,
        -2.556152343750000000000000000000e-01,
        -1.670837402343750000000000000000e-02,
        -3.765869140625000000000000000000e-02,
         7.663726806640625000000000000000e-03,
        -7.623291015625000000000000000000e-02,
         9.15

tensor([-3.877777233719825744628906250000e-02,
         5.772535502910614013671875000000e-02,
         1.810162216424942016601562500000e-01,
        -6.856413930654525756835937500000e-02,
         4.239537566900253295898437500000e-02,
        -6.748349219560623168945312500000e-02,
         2.082413993775844573974609375000e-02,
        -1.169816404581069946289062500000e-01,
        -2.263372093439102172851562500000e-01,
         1.134629640728235244750976562500e-02,
        -7.319548726081848144531250000000e-02,
        -5.384112708270549774169921875000e-03,
         9.074814617633819580078125000000e-03,
        -2.732892148196697235107421875000e-02,
        -1.855772882699966430664062500000e-01,
        -6.858005374670028686523437500000e-02,
         5.003200173377990722656250000000e-01,
        -7.989417016506195068359375000000e-02,
        -6.583850085735321044921875000000e-02,
        -2.003953605890274047851562500000e-01,
        -9.945812076330184936523437500000e-02,
         6.77

tensor([-1.756750792264938354492187500000e-01,
        -1.366655230522155761718750000000e-01,
        -4.617188870906829833984375000000e-02,
        -1.848238557577133178710937500000e-01,
         7.682895660400390625000000000000e-02,
        -4.752013087272644042968750000000e-01,
         9.281260520219802856445312500000e-02,
        -2.770404145121574401855468750000e-02,
        -2.576721273362636566162109375000e-02,
         1.701738089323043823242187500000e-01,
         5.908828973770141601562500000000e-02,
        -2.831601500511169433593750000000e-01,
        -2.855448983609676361083984375000e-02,
        -1.157951578497886657714843750000e-01,
        -3.516547083854675292968750000000e-01,
        -2.859713733196258544921875000000e-01,
         6.857888102531433105468750000000e-01,
        -4.427797719836235046386718750000e-02,
        -1.363292932510375976562500000000e-01,
        -1.881218552589416503906250000000e-01,
        -1.645136922597885131835937500000e-01,
        -6.79

In [189]:
def process_df(df):
    benford_d1 = [np.log10(1 + 1 / 1), np.log10(1 + 1 / 2), np.log10(1 + 1 / 3), np.log10(1 + 1 / 4),
                  np.log10(1 + 1 / 5), np.log10(1 + 1 / 6), np.log10(1 + 1 / 7), np.log10(1 + 1 / 8),
                  np.log10(1 + 1 / 9)]

    df['sumA'] = df['1'] + df['2'] + df['3'] + df['4'] + df['5'] + df['6'] + df['7'] + df['8'] + df['9']
    df['sumB'] = df['-1'] + df['-2'] + df['-3'] + df['-4'] + df['-5'] + df['-6'] + df['-7'] + df['-8'] + df['-9']
    df['sumC'] = df['sumA'] + df['sumB']

    df['log10(parameters)'] = np.log10(df['sumC'])

    df['parameter_category'] = np.int8(np.rint(df['log10(parameters)']))

    df['positive_ratio'] = df['sumA'] / df['sumC']
    df['negative_ratio'] = df['sumB'] / df['sumC']

    df['percent_zero'] = (df['0'] + df['-0']) / (df['sumC'] + df['0'] + df['-0'])

    for z in ['-1', '-2', '-3', '-4', '-5', '-6', '-7', '-8', '-9']:
        df[f'percent_neg{z}'] = df[z] / df['sumB']

        df.loc[df[f'percent_neg{z}'] == 0.0, f'percent_neg{z}'] = 0.00001
        df.fillna(0.00001, inplace=True)

    for z in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
        df[f'percent_{z}'] = df[z] / df['sumA']
        df[f'combined_{z}'] = (df[z] + df['-' + z]) / df['sumC']

        df.loc[df[f'percent_{z}'] == 0.0, f'percent_{z}'] = 0.00001

        df.loc[df[f'combined_{z}'] == 0.0, f'combined_{z}'] = 0.00001

        df.fillna(0.00001, inplace=True)

        if z != '0':
            percent_expected = np.log10(1 + (1 / int(z)))
            df[f'diffpercent_{z}'] = percent_expected - df[f'percent_{z}']
            df[f'diffpercent_neg{z}'] = percent_expected - df[f'percent_neg-{z}']

            print(percent_expected)
            df[f'diffcombined_{z}'] = percent_expected - df[f'combined_{z}']

    df['KL_relative_entropy'] = (df['combined_1'] * np.log(df['combined_1'] / benford_d1[0])) + \
                                (df['combined_2'] * np.log(df['combined_2'] / benford_d1[1])) + \
                                (df['combined_3'] * np.log(df['combined_3'] / benford_d1[2])) + \
                                (df['combined_4'] * np.log(df['combined_4'] / benford_d1[3])) + \
                                (df['combined_5'] * np.log(df['combined_5'] / benford_d1[4])) + \
                                (df['combined_6'] * np.log(df['combined_6'] / benford_d1[5])) + \
                                (df['combined_7'] * np.log(df['combined_7'] / benford_d1[6])) + \
                                (df['combined_8'] * np.log(df['combined_8'] / benford_d1[7])) + \
                                (df['combined_9'] * np.log(df['combined_9'] / benford_d1[8]))

    df['KL_relative_entropy_neg'] = (df['percent_neg-1'] * np.log(df['percent_neg-1'] / benford_d1[0])) + \
                                    (df['percent_neg-2'] * np.log(df['percent_neg-2'] / benford_d1[1])) + \
                                    (df['percent_neg-3'] * np.log(df['percent_neg-3'] / benford_d1[2])) + \
                                    (df['percent_neg-4'] * np.log(df['percent_neg-4'] / benford_d1[3])) + \
                                    (df['percent_neg-5'] * np.log(df['percent_neg-5'] / benford_d1[4])) + \
                                    (df['percent_neg-6'] * np.log(df['percent_neg-6'] / benford_d1[5])) + \
                                    (df['percent_neg-7'] * np.log(df['percent_neg-7'] / benford_d1[6])) + \
                                    (df['percent_neg-8'] * np.log(df['percent_neg-8'] / benford_d1[7])) + \
                                    (df['percent_neg-9'] * np.log(df['percent_neg-9'] / benford_d1[8]))

    df['KL_relative_entropy_pos'] = (df['percent_1'] * np.log(df['percent_1'] / benford_d1[0])) + \
                                    (df['percent_2'] * np.log(df['percent_2'] / benford_d1[1])) + \
                                    (df['percent_3'] * np.log(df['percent_3'] / benford_d1[2])) + \
                                    (df['percent_4'] * np.log(df['percent_4'] / benford_d1[3])) + \
                                    (df['percent_5'] * np.log(df['percent_5'] / benford_d1[4])) + \
                                    (df['percent_6'] * np.log(df['percent_6'] / benford_d1[5])) + \
                                    (df['percent_7'] * np.log(df['percent_7'] / benford_d1[6])) + \
                                    (df['percent_8'] * np.log(df['percent_8'] / benford_d1[7])) + \
                                    (df['percent_9'] * np.log(df['percent_9'] / benford_d1[8]))

    uniform = 1. / 9.

    df['KL_relative_entropy_uniform'] = (df['combined_1'] * np.log(df['combined_1'] / uniform)) + \
                                        (df['combined_2'] * np.log(df['combined_2'] / uniform)) + \
                                        (df['combined_3'] * np.log(df['combined_3'] / uniform)) + \
                                        (df['combined_4'] * np.log(df['combined_4'] / uniform)) + \
                                        (df['combined_5'] * np.log(df['combined_5'] / uniform)) + \
                                        (df['combined_6'] * np.log(df['combined_6'] / uniform)) + \
                                        (df['combined_7'] * np.log(df['combined_7'] / uniform)) + \
                                        (df['combined_8'] * np.log(df['combined_8'] / uniform)) + \
                                        (df['combined_9'] * np.log(df['combined_9'] / uniform)) \
 
    df['KL_relative_entropy_neg_uniform'] = (df['percent_neg-1'] * np.log(df['percent_neg-1'] / uniform)) + \
                                                    (df['percent_neg-2'] * np.log(df['percent_neg-2'] / uniform)) + \
                                                    (df['percent_neg-3'] * np.log(df['percent_neg-3'] / uniform)) + \
                                                    (df['percent_neg-4'] * np.log(df['percent_neg-4'] / uniform)) + \
                                                    (df['percent_neg-5'] * np.log(df['percent_neg-5'] / uniform)) + \
                                                    (df['percent_neg-6'] * np.log(df['percent_neg-6'] / uniform)) + \
                                                    (df['percent_neg-7'] * np.log(df['percent_neg-7'] / uniform)) + \
                                                    (df['percent_neg-8'] * np.log(df['percent_neg-8'] / uniform)) + \
                                                    (df['percent_neg-9'] * np.log(df['percent_neg-9'] / uniform)) \
 
    df['KL_relative_entropy_pos_uniform'] = (df['percent_1'] * np.log(df['percent_1'] / uniform)) + \
                                                    (df['percent_2'] * np.log(df['percent_2'] / uniform)) + \
                                                    (df['percent_3'] * np.log(df['percent_3'] / uniform)) + \
                                                    (df['percent_4'] * np.log(df['percent_4'] / uniform)) + \
                                                    (df['percent_5'] * np.log(df['percent_5'] / uniform)) + \
                                                    (df['percent_6'] * np.log(df['percent_6'] / uniform)) + \
                                                    (df['percent_7'] * np.log(df['percent_7'] / uniform)) + \
                                                    (df['percent_8'] * np.log(df['percent_8'] / uniform)) + \
                                                    (df['percent_9'] * np.log(df['percent_9'] / uniform))

    df['KL_benford_diff'] = df['KL_relative_entropy'].diff()
    df['KL_uniform_diff'] = df['KL_relative_entropy_uniform'].diff()

    df['accuracy_diff'] = df['accuracy'].diff()
    df['loss_diff'] = df['loss'].diff()
    df['lr'] = 0.10
    df['neg_ratio_diff'] = df['negative_ratio'].diff()

    return df

In [70]:
torch.set_printoptions(precision=30)
counter=0
for p in model.visual.parameters():
    print(p.dtype)
    counter+=1
    
counter

torch.float32
torch.float32
torch.float16
torch.float16
torch.float32
torch.float32
torch.float16
torch.float16
torch.float16
torch.float16
torch.float32
torch.float32
torch.float16
torch.float16
torch.float16
torch.float16
torch.float32
torch.float32
torch.float16
torch.float16
torch.float16
torch.float16
torch.float32
torch.float32
torch.float16
torch.float16
torch.float16
torch.float16
torch.float32
torch.float32
torch.float16
torch.float16
torch.float16
torch.float16
torch.float32
torch.float32
torch.float16
torch.float16
torch.float16
torch.float16
torch.float32
torch.float32
torch.float16
torch.float16
torch.float16
torch.float16
torch.float32
torch.float32
torch.float16
torch.float16
torch.float16
torch.float16
torch.float32
torch.float32
torch.float16
torch.float16
torch.float16
torch.float16
torch.float32
torch.float32
torch.float16
torch.float16
torch.float16
torch.float16
torch.float32
torch.float32
torch.float16
torch.float16
torch.float16
torch.float16
torch.float32
torch.

152

In [201]:
counter=0
model_name=[]
layer_number=[]
model_part =[]
layer_name =[]
layer_sum = []
layer_dtype = []
layer_mean=[]
layer_median=[]
layer_std=[]
layer_prod=[]
layer_var=[]
layer_nonzero=[]
layer_num_elements=[]
layer_max=[]
layer_min=[]
layer_shape=[]
layer_requires_grad=[]
layer_dim = []
later_element_size=[]

level_counter_list = []
level_counter2_list = []
expo_list=[]


for name, param in model.visual.named_parameters():
    with torch.no_grad():
        layer_number.append(counter)
        #print(name,param.dtype,torch.sum(param).cpu().numpy(),torch.mean(param).cpu().numpy(),torch.median(param).cpu().numpy(),
        #      torch.std(param).cpu().numpy(),torch.prod(param).cpu().numpy(),
        #     torch.var(param).cpu().numpy(),torch.count_nonzero(param).cpu().numpy(),
        #      torch.max(param).cpu().numpy(),
        #      torch.min(param).cpu().numpy(),param.shape,param.requires_grad,param.data.dim(),
         #     torch.numel(param.data))
        
        model_name.append('ViT-B/32')
        model_part.append('visual')
        layer_name.append(name)
        layer_number.append(i)
        layer_sum.append(torch.sum(param).cpu().numpy())
        layer_dtype.append(param.dtype)
        layer_mean.append(torch.mean(param).cpu().numpy())
        layer_median.append(torch.median(param).cpu().numpy())
        layer_std.append(torch.std(param).cpu().numpy())
        layer_prod.append(torch.prod(param).cpu().numpy())
        layer_var.append(torch.var(param).cpu().numpy())
        layer_nonzero.append(torch.count_nonzero(param).cpu().numpy())
        layer_num_elements.append(torch.numel(param.data))
        layer_max.append(torch.max(param).cpu().numpy())
        layer_min.append(torch.min(param).cpu().numpy())
        layer_shape.append(param.shape)
        layer_requires_grad.append(param.requires_grad)
        layer_dim.append(param.data.dim())
        
        level_counter = {'model':'ViT-B/32','part':'visual', 'layer': name, 'i':counter,'0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0,
                         '8': 0, '9': 0, '-0': 0, '-1': 0, '-2': 0, '-3': 0, '-4': 0, '-5': 0, '-6': 0, '-7': 0,
                         '-8': 0, '-9': 0}
        level_counter2 = {'model':'ViT-B/32','part':'visual', 'layer': name,'i':counter, '0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0,
                          '6': 0, '8': 0, '9': 0}
        
        
        lvl_exponent = {'model':'ViT-B/32','part':'visual', 'layer': name, 'i':counter,'00': 0, '+01': 0, '+02': 0, '+03': 0, '+04': 0, '+05': 0, '+06': 0, '+07': 0,
                         '+08': 0, '+09': 0, '+00': 0, '-00': 0, '-01': 0, '-02': 0, '-03': 0, '-04': 0, '-05': 0, '-06': 0, '-07': 0,
                         '-08': 0, '-09': 0,'-10':0,'-11':0,'-12':0,'-13':0,'-14':0,'-15':0,'-16':0,'-17':0,'-18':0,'-19':0,'-20':0,'-21':0,'-22':0,'-23':0,'-24':0}
        
        
        
        np_param = param.cpu().detach().numpy().flatten()
        #print(f'len(np_param)={len(np_param)}')

        reps = len(np_param) // 10000

        if len(np_param) % 10000 != 0:
            reps += 1

        for j in range(reps):
            start = j * 10000
            end = start + 10000
            if j == reps - 1:
                section = np_param[start:]
                # print(f'len(section)={len(section)} reps={reps} j={j} npparamlen={len(np_param)}')
            else:
                section = np_param[start:end]

            digits = np.array2string(section, formatter={'float_kind': lambda x: ("%0.1E" % x).split('E')[0]},
                                     threshold=1000000000)
            
            exponent = np.array2string(section, formatter={'float_kind': lambda x: ("%0.1E" % x).split('E')[1]},
                                     threshold=1000000000)

            if len(digits) > 0:
                for d in digits.split(" "):
                    digit = d.replace("\n", "").replace('[', '').replace(']', '')
                    fdigit = digit.split(".")[0]
                    sdigit = digit.split(".")[1]
                    level_counter[fdigit] += 1
                    level_counter2[sdigit] += 1

            if len(exponent) > 0:
                for d in exponent.split(" "):
                    digit = d.replace("\n", "").replace('[', '').replace(']', '')
                    #print(digit)
                    lvl_exponent[digit] += 1
                   


        level_counter_list.append(level_counter)
        level_counter2_list.append(level_counter2)
        expo_list.append(lvl_exponent)
        
        
        counter+=1
    
counter

counter=0


for name, param in model.token_embedding.named_parameters():
    with torch.no_grad():
        layer_number.append(counter)
        #print(name,param.dtype,torch.sum(param).cpu().numpy(),torch.mean(param).cpu().numpy(),torch.median(param).cpu().numpy(),
           #   torch.std(param).cpu().numpy(),torch.prod(param).cpu().numpy(),
            # torch.var(param).cpu().numpy(),torch.count_nonzero(param).cpu().numpy(),
           #   torch.max(param).cpu().numpy(),
           #   torch.min(param).cpu().numpy(),param.shape,param.requires_grad,param.data.dim(),
           #   torch.numel(param.data))
        
        model_name.append('ViT-B/32')
        model_part.append('token_embedding')
        layer_name.append(name)
        layer_number.append(i)
        layer_sum.append(torch.sum(param).cpu().numpy())
        layer_dtype.append(param.dtype)
        layer_mean.append(torch.mean(param).cpu().numpy())
        layer_median.append(torch.median(param).cpu().numpy())
        layer_std.append(torch.std(param).cpu().numpy())
        layer_prod.append(torch.prod(param).cpu().numpy())
        layer_var.append(torch.var(param).cpu().numpy())
        layer_nonzero.append(torch.count_nonzero(param).cpu().numpy())
        layer_num_elements.append(torch.numel(param.data))
        layer_max.append(torch.max(param).cpu().numpy())
        layer_min.append(torch.min(param).cpu().numpy())
        layer_shape.append(param.shape)
        layer_requires_grad.append(param.requires_grad)
        layer_dim.append(param.data.dim())
        
        level_counter = {'model':'ViT-B/32','part':'token_embedding', 'layer': name,'i':counter, '0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0,
                         '8': 0, '9': 0, '-0': 0, '-1': 0, '-2': 0, '-3': 0, '-4': 0, '-5': 0, '-6': 0, '-7': 0,
                         '-8': 0, '-9': 0}
        level_counter2 = {'model':'ViT-B/32','part':'token_embedding', 'layer': name,'i':counter, '0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0,
                          '6': 0, '8': 0, '9': 0}
        
        
        lvl_exponent = {'model':'ViT-B/32','part':'token_embedding', 'layer': name, 'i':counter,'00': 0, '+01': 0, '+02': 0, '+03': 0, '+04': 0, '+05': 0, '+06': 0, '+07': 0,
                         '+08': 0, '+09': 0, '+00': 0, '-00': 0, '-01': 0, '-02': 0, '-03': 0, '-04': 0, '-05': 0, '-06': 0, '-07': 0,
                         '-08': 0, '-09': 0,'-10':0,'-11':0,'-12':0,'-13':0,'-14':0,'-15':0,'-16':0,'-17':0,'-18':0,'-19':0,'-20':0,'-21':0,'-22':0,'-23':0,'-24':0}
        
        np_param = param.cpu().detach().numpy().flatten()
        #print(f'len(np_param)={len(np_param)}')

        reps = len(np_param) // 10000

        if len(np_param) % 10000 != 0:
            reps += 1

        for j in range(reps):
            start = j * 10000
            end = start + 10000
            if j == reps - 1:
                section = np_param[start:]
                # print(f'len(section)={len(section)} reps={reps} j={j} npparamlen={len(np_param)}')
            else:
                section = np_param[start:end]

            digits = np.array2string(section, formatter={'float_kind': lambda x: ("%0.1E" % x).split('E')[0]},
                                     threshold=1000000000)
            
            exponent = np.array2string(section, formatter={'float_kind': lambda x: ("%0.1E" % x).split('E')[1]},
                                     threshold=1000000000)

            if len(digits) > 0:
                for d in digits.split(" "):
                    digit = d.replace("\n", "").replace('[', '').replace(']', '')
                    fdigit = digit.split(".")[0]
                    sdigit = digit.split(".")[1]
                    level_counter[fdigit] += 1
                    level_counter2[sdigit] += 1
            
            if len(exponent) > 0:
                for d in exponent.split(" "):
                    digit = d.replace("\n", "").replace('[', '').replace(']', '')
                    
                    lvl_exponent[digit] += 1



        level_counter_list.append(level_counter)
        level_counter2_list.append(level_counter2)
        expo_list.append(lvl_exponent)
        counter+=1
    
counter

counter=0


for name, param in model.ln_final.named_parameters():
    with torch.no_grad():
        layer_number.append(counter)
        #print(name,param.dtype,torch.sum(param).cpu().numpy(),torch.mean(param).cpu().numpy(),torch.median(param).cpu().numpy(),
            #  torch.std(param).cpu().numpy(),torch.prod(param).cpu().numpy(),
          #   torch.var(param).cpu().numpy(),torch.count_nonzero(param).cpu().numpy(),
          #    torch.max(param).cpu().numpy(),
         #     torch.min(param).cpu().numpy(),param.shape,param.requires_grad,param.data.dim(),
         #     torch.numel(param.data))
        
        model_name.append('ViT-B/32')
        model_part.append('ln_final')
        layer_name.append(name)
        layer_number.append(i)
        layer_sum.append(torch.sum(param).cpu().numpy())
        layer_dtype.append(param.dtype)
        layer_mean.append(torch.mean(param).cpu().numpy())
        layer_median.append(torch.median(param).cpu().numpy())
        layer_std.append(torch.std(param).cpu().numpy())
        layer_prod.append(torch.prod(param).cpu().numpy())
        layer_var.append(torch.var(param).cpu().numpy())
        layer_nonzero.append(torch.count_nonzero(param).cpu().numpy())
        layer_num_elements.append(torch.numel(param.data))
        layer_max.append(torch.max(param).cpu().numpy())
        layer_min.append(torch.min(param).cpu().numpy())
        layer_shape.append(param.shape)
        layer_requires_grad.append(param.requires_grad)
        layer_dim.append(param.data.dim())
        
        level_counter = {'model':'ViT-B/32','part':'ln_final', 'layer': name,'i':counter, '0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0,
                         '8': 0, '9': 0, '-0': 0, '-1': 0, '-2': 0, '-3': 0, '-4': 0, '-5': 0, '-6': 0, '-7': 0,
                         '-8': 0, '-9': 0}
        level_counter2 = {'model':'ViT-B/32','part':'ln_final', 'layer': name,'i':counter, '0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0,
                          '6': 0, '8': 0, '9': 0}
        
        lvl_exponent = {'model':'ViT-B/32','part':'ln_final', 'layer': name, 'i':counter,'00': 0, '+01': 0, '+02': 0, '+03': 0, '+04': 0, '+05': 0, '+06': 0, '+07': 0,
                         '+08': 0, '+09': 0, '+00': 0, '-00': 0, '-01': 0, '-02': 0, '-03': 0, '-04': 0, '-05': 0, '-06': 0, '-07': 0,
                         '-08': 0, '-09': 0,'-10':0,'-11':0,'-12':0,'-13':0,'-14':0,'-15':0,'-16':0,'-17':0,'-18':0,'-19':0,'-20':0,'-21':0,'-22':0,'-23':0,'-24':0}
        
        
        np_param = param.cpu().detach().numpy().flatten()
        #print(f'len(np_param)={len(np_param)}')

        reps = len(np_param) // 10000

        if len(np_param) % 10000 != 0:
            reps += 1

        for j in range(reps):
            start = j * 10000
            end = start + 10000
            if j == reps - 1:
                section = np_param[start:]
                # print(f'len(section)={len(section)} reps={reps} j={j} npparamlen={len(np_param)}')
            else:
                section = np_param[start:end]

            digits = np.array2string(section, formatter={'float_kind': lambda x: ("%0.1E" % x).split('E')[0]},
                                     threshold=1000000000)
            
            exponent = np.array2string(section, formatter={'float_kind': lambda x: ("%0.1E" % x).split('E')[1]},
                                     threshold=1000000000)

            if len(digits) > 0:
                for d in digits.split(" "):
                    digit = d.replace("\n", "").replace('[', '').replace(']', '')
                    fdigit = digit.split(".")[0]
                    sdigit = digit.split(".")[1]
                    level_counter[fdigit] += 1
                    level_counter2[sdigit] += 1
                    
            
         
                    
            if len(exponent) > 0:
                for d in exponent.split(" "):
                    digit = d.replace("\n", "").replace('[', '').replace(']', '')
                    
                    lvl_exponent[digit] += 1



        level_counter_list.append(level_counter)
        level_counter2_list.append(level_counter2)
        expo_list.append(lvl_exponent)
        counter+=1
    
counter

counter=0




df = pd.DataFrame({'model_name':model_name,
                 'layer_number':layer_number,
                  'model_part':model_part,
                  'i':layer_number,
                 'layer_name':layer_name,
                  'layer_dtype':layer_dtype,
                  'layer_nonzero':layer_nonzero,
                  'layer_num_elements':layer_num_elements,
                  'layer_sum':layer_sum,
                 'layer_mean':layer_mean,
                 'layer_std':layer_std,
                  'layer_median':layer_median,
                 'layer_prod':layer_prod,
                  'layer_var':layer_var,
                  'layer_max':layer_max,
                  'layer_min':layer_min,
                  'layer_shape':layer_shape,
                  'layer_requires_grad':layer_requires_grad,
                  'layer_dim':layer_dim,
                  'later_element_size':later_element_size}
                  
                 )



df.to_csv('modinfo.csv')


df = pd.DataFrame(level_counter_list)
df.to_csv(f'lvl_counter_ViT.csv')

df = pd.DataFrame(level_counter2_list)
df.to_csv(f'lvl_counter2_ViT.csv')



df = pd.DataFrame(expo_list)
df.to_csv(f'expo_ViT.csv')



ValueError: arrays must all be same length

In [52]:
imagenet_classes = ["tench", "goldfish", "great white shark", "tiger shark", "hammerhead shark", "electric ray", "stingray", "rooster", "hen", "ostrich", "brambling", "goldfinch", "house finch", "junco", "indigo bunting", "American robin", "bulbul", "jay", "magpie", "chickadee", "American dipper", "kite (bird of prey)", "bald eagle", "vulture", "great grey owl", "fire salamander", "smooth newt", "newt", "spotted salamander", "axolotl", "American bullfrog", "tree frog", "tailed frog", "loggerhead sea turtle", "leatherback sea turtle", "mud turtle", "terrapin", "box turtle", "banded gecko", "green iguana", "Carolina anole", "desert grassland whiptail lizard", "agama", "frilled-necked lizard", "alligator lizard", "Gila monster", "European green lizard", "chameleon", "Komodo dragon", "Nile crocodile", "American alligator", "triceratops", "worm snake", "ring-necked snake", "eastern hog-nosed snake", "smooth green snake", "kingsnake", "garter snake", "water snake", "vine snake", "night snake", "boa constrictor", "African rock python", "Indian cobra", "green mamba", "sea snake", "Saharan horned viper", "eastern diamondback rattlesnake", "sidewinder rattlesnake", "trilobite", "harvestman", "scorpion", "yellow garden spider", "barn spider", "European garden spider", "southern black widow", "tarantula", "wolf spider", "tick", "centipede", "black grouse", "ptarmigan", "ruffed grouse", "prairie grouse", "peafowl", "quail", "partridge", "african grey parrot", "macaw", "sulphur-crested cockatoo", "lorikeet", "coucal", "bee eater", "hornbill", "hummingbird", "jacamar", "toucan", "duck", "red-breasted merganser", "goose", "black swan", "tusker", "echidna", "platypus", "wallaby", "koala", "wombat", "jellyfish", "sea anemone", "brain coral", "flatworm", "nematode", "conch", "snail", "slug", "sea slug", "chiton", "chambered nautilus", "Dungeness crab", "rock crab", "fiddler crab", "red king crab", "American lobster", "spiny lobster", "crayfish", "hermit crab", "isopod", "white stork", "black stork", "spoonbill", "flamingo", "little blue heron", "great egret", "bittern bird", "crane bird", "limpkin", "common gallinule", "American coot", "bustard", "ruddy turnstone", "dunlin", "common redshank", "dowitcher", "oystercatcher", "pelican", "king penguin", "albatross", "grey whale", "killer whale", "dugong", "sea lion", "Chihuahua", "Japanese Chin", "Maltese", "Pekingese", "Shih Tzu", "King Charles Spaniel", "Papillon", "toy terrier", "Rhodesian Ridgeback", "Afghan Hound", "Basset Hound", "Beagle", "Bloodhound", "Bluetick Coonhound", "Black and Tan Coonhound", "Treeing Walker Coonhound", "English foxhound", "Redbone Coonhound", "borzoi", "Irish Wolfhound", "Italian Greyhound", "Whippet", "Ibizan Hound", "Norwegian Elkhound", "Otterhound", "Saluki", "Scottish Deerhound", "Weimaraner", "Staffordshire Bull Terrier", "American Staffordshire Terrier", "Bedlington Terrier", "Border Terrier", "Kerry Blue Terrier", "Irish Terrier", "Norfolk Terrier", "Norwich Terrier", "Yorkshire Terrier", "Wire Fox Terrier", "Lakeland Terrier", "Sealyham Terrier", "Airedale Terrier", "Cairn Terrier", "Australian Terrier", "Dandie Dinmont Terrier", "Boston Terrier", "Miniature Schnauzer", "Giant Schnauzer", "Standard Schnauzer", "Scottish Terrier", "Tibetan Terrier", "Australian Silky Terrier", "Soft-coated Wheaten Terrier", "West Highland White Terrier", "Lhasa Apso", "Flat-Coated Retriever", "Curly-coated Retriever", "Golden Retriever", "Labrador Retriever", "Chesapeake Bay Retriever", "German Shorthaired Pointer", "Vizsla", "English Setter", "Irish Setter", "Gordon Setter", "Brittany dog", "Clumber Spaniel", "English Springer Spaniel", "Welsh Springer Spaniel", "Cocker Spaniel", "Sussex Spaniel", "Irish Water Spaniel", "Kuvasz", "Schipperke", "Groenendael dog", "Malinois", "Briard", "Australian Kelpie", "Komondor", "Old English Sheepdog", "Shetland Sheepdog", "collie", "Border Collie", "Bouvier des Flandres dog", "Rottweiler", "German Shepherd Dog", "Dobermann", "Miniature Pinscher", "Greater Swiss Mountain Dog", "Bernese Mountain Dog", "Appenzeller Sennenhund", "Entlebucher Sennenhund", "Boxer", "Bullmastiff", "Tibetan Mastiff", "French Bulldog", "Great Dane", "St. Bernard", "husky", "Alaskan Malamute", "Siberian Husky", "Dalmatian", "Affenpinscher", "Basenji", "pug", "Leonberger", "Newfoundland dog", "Great Pyrenees dog", "Samoyed", "Pomeranian", "Chow Chow", "Keeshond", "brussels griffon", "Pembroke Welsh Corgi", "Cardigan Welsh Corgi", "Toy Poodle", "Miniature Poodle", "Standard Poodle", "Mexican hairless dog (xoloitzcuintli)", "grey wolf", "Alaskan tundra wolf", "red wolf or maned wolf", "coyote", "dingo", "dhole", "African wild dog", "hyena", "red fox", "kit fox", "Arctic fox", "grey fox", "tabby cat", "tiger cat", "Persian cat", "Siamese cat", "Egyptian Mau", "cougar", "lynx", "leopard", "snow leopard", "jaguar", "lion", "tiger", "cheetah", "brown bear", "American black bear", "polar bear", "sloth bear", "mongoose", "meerkat", "tiger beetle", "ladybug", "ground beetle", "longhorn beetle", "leaf beetle", "dung beetle", "rhinoceros beetle", "weevil", "fly", "bee", "ant", "grasshopper", "cricket insect", "stick insect", "cockroach", "praying mantis", "cicada", "leafhopper", "lacewing", "dragonfly", "damselfly", "red admiral butterfly", "ringlet butterfly", "monarch butterfly", "small white butterfly", "sulphur butterfly", "gossamer-winged butterfly", "starfish", "sea urchin", "sea cucumber", "cottontail rabbit", "hare", "Angora rabbit", "hamster", "porcupine", "fox squirrel", "marmot", "beaver", "guinea pig", "common sorrel horse", "zebra", "pig", "wild boar", "warthog", "hippopotamus", "ox", "water buffalo", "bison", "ram (adult male sheep)", "bighorn sheep", "Alpine ibex", "hartebeest", "impala (antelope)", "gazelle", "arabian camel", "llama", "weasel", "mink", "European polecat", "black-footed ferret", "otter", "skunk", "badger", "armadillo", "three-toed sloth", "orangutan", "gorilla", "chimpanzee", "gibbon", "siamang", "guenon", "patas monkey", "baboon", "macaque", "langur", "black-and-white colobus", "proboscis monkey", "marmoset", "white-headed capuchin", "howler monkey", "titi monkey", "Geoffroy's spider monkey", "common squirrel monkey", "ring-tailed lemur", "indri", "Asian elephant", "African bush elephant", "red panda", "giant panda", "snoek fish", "eel", "silver salmon", "rock beauty fish", "clownfish", "sturgeon", "gar fish", "lionfish", "pufferfish", "abacus", "abaya", "academic gown", "accordion", "acoustic guitar", "aircraft carrier", "airliner", "airship", "altar", "ambulance", "amphibious vehicle", "analog clock", "apiary", "apron", "trash can", "assault rifle", "backpack", "bakery", "balance beam", "balloon", "ballpoint pen", "Band-Aid", "banjo", "baluster / handrail", "barbell", "barber chair", "barbershop", "barn", "barometer", "barrel", "wheelbarrow", "baseball", "basketball", "bassinet", "bassoon", "swimming cap", "bath towel", "bathtub", "station wagon", "lighthouse", "beaker", "military hat (bearskin or shako)", "beer bottle", "beer glass", "bell tower", "baby bib", "tandem bicycle", "bikini", "ring binder", "binoculars", "birdhouse", "boathouse", "bobsleigh", "bolo tie", "poke bonnet", "bookcase", "bookstore", "bottle cap", "hunting bow", "bow tie", "brass memorial plaque", "bra", "breakwater", "breastplate", "broom", "bucket", "buckle", "bulletproof vest", "high-speed train", "butcher shop", "taxicab", "cauldron", "candle", "cannon", "canoe", "can opener", "cardigan", "car mirror", "carousel", "tool kit", "cardboard box / carton", "car wheel", "automated teller machine", "cassette", "cassette player", "castle", "catamaran", "CD player", "cello", "mobile phone", "chain", "chain-link fence", "chain mail", "chainsaw", "storage chest", "chiffonier", "bell or wind chime", "china cabinet", "Christmas stocking", "church", "movie theater", "cleaver", "cliff dwelling", "cloak", "clogs", "cocktail shaker", "coffee mug", "coffeemaker", "spiral or coil", "combination lock", "computer keyboard", "candy store", "container ship", "convertible", "corkscrew", "cornet", "cowboy boot", "cowboy hat", "cradle", "construction crane", "crash helmet", "crate", "infant bed", "Crock Pot", "croquet ball", "crutch", "cuirass", "dam", "desk", "desktop computer", "rotary dial telephone", "diaper", "digital clock", "digital watch", "dining table", "dishcloth", "dishwasher", "disc brake", "dock", "dog sled", "dome", "doormat", "drilling rig", "drum", "drumstick", "dumbbell", "Dutch oven", "electric fan", "electric guitar", "electric locomotive", "entertainment center", "envelope", "espresso machine", "face powder", "feather boa", "filing cabinet", "fireboat", "fire truck", "fire screen", "flagpole", "flute", "folding chair", "football helmet", "forklift", "fountain", "fountain pen", "four-poster bed", "freight car", "French horn", "frying pan", "fur coat", "garbage truck", "gas mask or respirator", "gas pump", "goblet", "go-kart", "golf ball", "golf cart", "gondola", "gong", "gown", "grand piano", "greenhouse", "radiator grille", "grocery store", "guillotine", "hair clip", "hair spray", "half-track", "hammer", "hamper", "hair dryer", "hand-held computer", "handkerchief", "hard disk drive", "harmonica", "harp", "combine harvester", "hatchet", "holster", "home theater", "honeycomb", "hook", "hoop skirt", "gymnastic horizontal bar", "horse-drawn vehicle", "hourglass", "iPod", "clothes iron", "carved pumpkin", "jeans", "jeep", "T-shirt", "jigsaw puzzle", "rickshaw", "joystick", "kimono", "knee pad", "knot", "lab coat", "ladle", "lampshade", "laptop computer", "lawn mower", "lens cap", "letter opener", "library", "lifeboat", "lighter", "limousine", "ocean liner", "lipstick", "slip-on shoe", "lotion", "music speaker", "loupe magnifying glass", "sawmill", "magnetic compass", "messenger bag", "mailbox", "tights", "one-piece bathing suit", "manhole cover", "maraca", "marimba", "mask", "matchstick", "maypole", "maze", "measuring cup", "medicine cabinet", "megalith", "microphone", "microwave oven", "military uniform", "milk can", "minibus", "miniskirt", "minivan", "missile", "mitten", "mixing bowl", "mobile home", "ford model t", "modem", "monastery", "monitor", "moped", "mortar and pestle", "graduation cap", "mosque", "mosquito net", "vespa", "mountain bike", "tent", "computer mouse", "mousetrap", "moving van", "muzzle", "metal nail", "neck brace", "necklace", "baby pacifier", "notebook computer", "obelisk", "oboe", "ocarina", "odometer", "oil filter", "pipe organ", "oscilloscope", "overskirt", "bullock cart", "oxygen mask", "product packet / packaging", "paddle", "paddle wheel", "padlock", "paintbrush", "pajamas", "palace", "pan flute", "paper towel", "parachute", "parallel bars", "park bench", "parking meter", "railroad car", "patio", "payphone", "pedestal", "pencil case", "pencil sharpener", "perfume", "Petri dish", "photocopier", "plectrum", "Pickelhaube", "picket fence", "pickup truck", "pier", "piggy bank", "pill bottle", "pillow", "ping-pong ball", "pinwheel", "pirate ship", "drink pitcher", "block plane", "planetarium", "plastic bag", "plate rack", "farm plow", "plunger", "Polaroid camera", "pole", "police van", "poncho", "pool table", "soda bottle", "plant pot", "potter's wheel", "power drill", "prayer rug", "printer", "prison", "missile", "projector", "hockey puck", "punching bag", "purse", "quill", "quilt", "race car", "racket", "radiator", "radio", "radio telescope", "rain barrel", "recreational vehicle", "fishing casting reel", "reflex camera", "refrigerator", "remote control", "restaurant", "revolver", "rifle", "rocking chair", "rotisserie", "eraser", "rugby ball", "ruler measuring stick", "sneaker", "safe", "safety pin", "salt shaker", "sandal", "sarong", "saxophone", "scabbard", "weighing scale", "school bus", "schooner", "scoreboard", "CRT monitor", "screw", "screwdriver", "seat belt", "sewing machine", "shield", "shoe store", "shoji screen / room divider", "shopping basket", "shopping cart", "shovel", "shower cap", "shower curtain", "ski", "balaclava ski mask", "sleeping bag", "slide rule", "sliding door", "slot machine", "snorkel", "snowmobile", "snowplow", "soap dispenser", "soccer ball", "sock", "solar thermal collector", "sombrero", "soup bowl", "keyboard space bar", "space heater", "space shuttle", "spatula", "motorboat", "spider web", "spindle", "sports car", "spotlight", "stage", "steam locomotive", "through arch bridge", "steel drum", "stethoscope", "scarf", "stone wall", "stopwatch", "stove", "strainer", "tram", "stretcher", "couch", "stupa", "submarine", "suit", "sundial", "sunglasses", "sunglasses", "sunscreen", "suspension bridge", "mop", "sweatshirt", "swim trunks / shorts", "swing", "electrical switch", "syringe", "table lamp", "tank", "tape player", "teapot", "teddy bear", "television", "tennis ball", "thatched roof", "front curtain", "thimble", "threshing machine", "throne", "tile roof", "toaster", "tobacco shop", "toilet seat", "torch", "totem pole", "tow truck", "toy store", "tractor", "semi-trailer truck", "tray", "trench coat", "tricycle", "trimaran", "tripod", "triumphal arch", "trolleybus", "trombone", "hot tub", "turnstile", "typewriter keyboard", "umbrella", "unicycle", "upright piano", "vacuum cleaner", "vase", "vaulted or arched ceiling", "velvet fabric", "vending machine", "vestment", "viaduct", "violin", "volleyball", "waffle iron", "wall clock", "wallet", "wardrobe", "military aircraft", "sink", "washing machine", "water bottle", "water jug", "water tower", "whiskey jug", "whistle", "hair wig", "window screen", "window shade", "Windsor tie", "wine bottle", "airplane wing", "wok", "wooden spoon", "wool", "split-rail fence", "shipwreck", "sailboat", "yurt", "website", "comic book", "crossword", "traffic or street sign", "traffic light", "dust jacket", "menu", "plate", "guacamole", "consomme", "hot pot", "trifle", "ice cream", "popsicle", "baguette", "bagel", "pretzel", "cheeseburger", "hot dog", "mashed potatoes", "cabbage", "broccoli", "cauliflower", "zucchini", "spaghetti squash", "acorn squash", "butternut squash", "cucumber", "artichoke", "bell pepper", "cardoon", "mushroom", "Granny Smith apple", "strawberry", "orange", "lemon", "fig", "pineapple", "banana", "jackfruit", "cherimoya (custard apple)", "pomegranate", "hay", "carbonara", "chocolate syrup", "dough", "meatloaf", "pizza", "pot pie", "burrito", "red wine", "espresso", "tea cup", "eggnog", "mountain", "bubble", "cliff", "coral reef", "geyser", "lakeshore", "promontory", "sandbar", "beach", "valley", "volcano", "baseball player", "bridegroom", "scuba diver", "rapeseed", "daisy", "yellow lady's slipper", "corn", "acorn", "rose hip", "horse chestnut seed", "coral fungus", "agaric", "gyromitra", "stinkhorn mushroom", "earth star fungus", "hen of the woods mushroom", "bolete", "corn cob", "toilet paper"]

A subset of these class names are modified from the default ImageNet class names sourced from Anish Athalye's imagenet-simple-labels.

These edits were made via trial and error and concentrated on the lowest performing classes according to top_1 and top_5 accuracy on the ImageNet training set for the RN50, RN101, and RN50x4 models. These tweaks improve top_1 by 1.5% on ViT-B/32 over using the default class names. Alec got bored somewhere along the way as gains started to diminish and never finished updating / tweaking the list. He also didn't revisit this with the better performing RN50x16, RN50x64, or any of the ViT models. He thinks it's likely another 0.5% to 1% top_1 could be gained from further work here. It'd be interesting to more rigorously study / understand this.

Some examples beyond the crane/crane -> construction crane / bird crane issue mentioned in Section 3.1.4 of the paper include:

- CLIP interprets "nail" as "fingernail" so we changed the label to "metal nail".
- ImageNet kite class refers to the bird of prey, not the flying toy, so we changed "kite" to "kite (bird of prey)"
- The ImageNet class for red wolf seems to include a lot of mislabeled maned wolfs so we changed "red wolf" to "red wolf or maned wolf"

In [53]:
imagenet_templates = [
    'a bad photo of a {}.',
    'a photo of many {}.',
    'a sculpture of a {}.',
    'a photo of the hard to see {}.',
    'a low resolution photo of the {}.',
    'a rendering of a {}.',
    'graffiti of a {}.',
    'a bad photo of the {}.',
    'a cropped photo of the {}.',
    'a tattoo of a {}.',
    'the embroidered {}.',
    'a photo of a hard to see {}.',
    'a bright photo of a {}.',
    'a photo of a clean {}.',
    'a photo of a dirty {}.',
    'a dark photo of the {}.',
    'a drawing of a {}.',
    'a photo of my {}.',
    'the plastic {}.',
    'a photo of the cool {}.',
    'a close-up photo of a {}.',
    'a black and white photo of the {}.',
    'a painting of the {}.',
    'a painting of a {}.',
    'a pixelated photo of the {}.',
    'a sculpture of the {}.',
    'a bright photo of the {}.',
    'a cropped photo of a {}.',
    'a plastic {}.',
    'a photo of the dirty {}.',
    'a jpeg corrupted photo of a {}.',
    'a blurry photo of the {}.',
    'a photo of the {}.',
    'a good photo of the {}.',
    'a rendering of the {}.',
    'a {} in a video game.',
    'a photo of one {}.',
    'a doodle of a {}.',
    'a close-up photo of the {}.',
    'a photo of a {}.',
    'the origami {}.',
    'the {} in a video game.',
    'a sketch of a {}.',
    'a doodle of the {}.',
    'a origami {}.',
    'a low resolution photo of a {}.',
    'the toy {}.',
    'a rendition of the {}.',
    'a photo of the clean {}.',
    'a photo of a large {}.',
    'a rendition of a {}.',
    'a photo of a nice {}.',
    'a photo of a weird {}.',
    'a blurry photo of a {}.',
    'a cartoon {}.',
    'art of a {}.',
    'a sketch of the {}.',
    'a embroidered {}.',
    'a pixelated photo of a {}.',
    'itap of the {}.',
    'a jpeg corrupted photo of the {}.',
    'a good photo of a {}.',
    'a plushie {}.',
    'a photo of the nice {}.',
    'a photo of the small {}.',
    'a photo of the weird {}.',
    'the cartoon {}.',
    'art of the {}.',
    'a drawing of the {}.',
    'a photo of the large {}.',
    'a black and white photo of a {}.',
    'the plushie {}.',
    'a dark photo of a {}.',
    'itap of a {}.',
    'graffiti of the {}.',
    'a toy {}.',
    'itap of my {}.',
    'a photo of a cool {}.',
    'a photo of a small {}.',
    'a tattoo of the {}.',
]

print(f"{len(imagenet_classes)} classes, {len(imagenet_templates)} templates")

1000 classes, 80 templates


A similar, intuition-guided trial and error based on the ImageNet training set was used for templates. This list is pretty haphazard and was gradually made / expanded over the course of about a year of the project and was revisited / tweaked every few months. A surprising / weird thing was adding templates intended to help ImageNet-R performance (specifying different possible renditions of an object) improved standard ImageNet accuracy too.

After the 80 templates were "locked" for the paper, we ran sequential forward selection over the list of 80 templates. The search terminated after ensembling 7 templates and selected them in the order below.

1. itap of a {}.
2. a bad photo of the {}.
3. a origami {}.
4. a photo of the large {}.
5. a {} in a video game.
6. art of the {}.
7. a photo of the small {}.

Speculating, we think it's interesting to see different scales (large and small), a difficult view (a bad photo), and "abstract" versions (origami, video game, art), were all selected for, but we haven't studied this in any detail. This subset performs a bit better than the full 80 ensemble reported in the paper, especially for the smaller models.

# Loading the Images

The ILSVRC2012 datasets are no longer available for download publicly. We instead download the ImageNet-V2 dataset by [Recht et al.](https://arxiv.org/abs/1902.10811).

If you have the ImageNet dataset downloaded, you can replace the dataset with the official torchvision loader, e.g.:

```python
images = torchvision.datasets.ImageNet("path/to/imagenet", split='val', transform=preprocess)
```

In [54]:
#! pip install git+https://github.com/modestyachts/ImageNetV2_pytorch

from imagenetv2_pytorch import ImageNetV2Dataset

images = ImageNetV2Dataset(transform=preprocess)
loader = torch.utils.data.DataLoader(images, batch_size=32, num_workers=16)

# Creating zero-shot classifier weights

In [55]:
def zeroshot_classifier(classnames, templates):
    with torch.no_grad():
        zeroshot_weights = []
        for classname in tqdm(classnames):
            texts = [template.format(classname) for template in templates] #format with class
            texts = clip.tokenize(texts).cuda() #tokenize
            class_embeddings = model.encode_text(texts) #embed with text encoder
            class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)
            class_embedding = class_embeddings.mean(dim=0)
            class_embedding /= class_embedding.norm()
            zeroshot_weights.append(class_embedding)
        zeroshot_weights = torch.stack(zeroshot_weights, dim=1).cuda()
    return zeroshot_weights


zeroshot_weights = zeroshot_classifier(imagenet_classes, imagenet_templates)

  0%|          | 0/1000 [00:00<?, ?it/s]

# Zero-shot prediction

In [56]:
def accuracy(output, target, topk=(1,)):
    pred = output.topk(max(topk), 1, True, True)[1].t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))
    return [float(correct[:k].reshape(-1).float().sum(0, keepdim=True).cpu().numpy()) for k in topk]

In [57]:
#ViT with 32 bit 
with torch.no_grad():
    top1, top5, n = 0., 0., 0.
    for i, (images, target) in enumerate(tqdm(loader)):
        images = images.cuda()
        target = target.cuda()
        
        # predict
        image_features = model.encode_image(images)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        logits = 100. * image_features @ zeroshot_weights

        # measure accuracy
        acc1, acc5 = accuracy(logits, target, topk=(1, 5))
        top1 += acc1
        top5 += acc5
        n += images.size(0)

top1 = (top1 / n) * 100
top5 = (top5 / n) * 100 

print(f"Top-1 accuracy: {top1:.2f}")
print(f"Top-5 accuracy: {top5:.2f}")

  0%|          | 0/313 [00:00<?, ?it/s]

Top-1 accuracy: 55.72
Top-5 accuracy: 83.47


In [47]:
#ViT with 16 bit 
with torch.no_grad():
    top1, top5, n = 0., 0., 0.
    for i, (images, target) in enumerate(tqdm(loader)):
        images = images.cuda()
        target = target.cuda()
        
        # predict
        image_features = model.encode_image(images)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        logits = 100. * image_features @ zeroshot_weights

        # measure accuracy
        acc1, acc5 = accuracy(logits, target, topk=(1, 5))
        top1 += acc1
        top5 += acc5
        n += images.size(0)

top1 = (top1 / n) * 100
top5 = (top5 / n) * 100 

print(f"Top-1 accuracy: {top1:.2f}")
print(f"Top-5 accuracy: {top5:.2f}")

  0%|          | 0/313 [00:00<?, ?it/s]

Top-1 accuracy: 55.72
Top-5 accuracy: 83.47


In [37]:
#RN50x4 with 32 bit 
with torch.no_grad():
    top1, top5, n = 0., 0., 0.
    for i, (images, target) in enumerate(tqdm(loader)):
        images = images.cuda()
        target = target.cuda()
        
        # predict
        image_features = model.encode_image(images)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        logits = 100. * image_features @ zeroshot_weights

        # measure accuracy
        acc1, acc5 = accuracy(logits, target, topk=(1, 5))
        top1 += acc1
        top5 += acc5
        n += images.size(0)

top1 = (top1 / n) * 100
top5 = (top5 / n) * 100 

print(f"Top-1 accuracy: {top1:.2f}")
print(f"Top-5 accuracy: {top5:.2f}")

  0%|          | 0/313 [00:00<?, ?it/s]

Top-1 accuracy: 59.35
Top-5 accuracy: 85.84


In [27]:
#RN50x4 with 32 bit 
with torch.no_grad():
    top1, top5, n = 0., 0., 0.
    for i, (images, target) in enumerate(tqdm(loader)):
        images = images.cuda()
        target = target.cuda()
        
        # predict
        image_features = model.encode_image(images)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        logits = 100. * image_features @ zeroshot_weights

        # measure accuracy
        acc1, acc5 = accuracy(logits, target, topk=(1, 5))
        top1 += acc1
        top5 += acc5
        n += images.size(0)

top1 = (top1 / n) * 100
top5 = (top5 / n) * 100 

print(f"Top-1 accuracy: {top1:.2f}")
print(f"Top-5 accuracy: {top5:.2f}")

  0%|          | 0/313 [00:00<?, ?it/s]

Top-1 accuracy: 59.35
Top-5 accuracy: 85.84


## 101


In [ ]:
with torch.no_grad():
    top1, top5, n = 0., 0., 0.
    for i, (images, target) in enumerate(tqdm(loader)):
        images = images.cuda()
        target = target.cuda()
        
        # predict
        image_features = model.encode_image(images)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        logits = 100. * image_features @ zeroshot_weights

        # measure accuracy
        acc1, acc5 = accuracy(logits, target, topk=(1, 5))
        top1 += acc1
        top5 += acc5
        n += images.size(0)

top1 = (top1 / n) * 100
top5 = (top5 / n) * 100 

print(f"Top-1 accuracy: {top1:.2f}")
print(f"Top-5 accuracy: {top5:.2f}")

In [96]:
import os
import clip
import torch
from torchvision.datasets import CIFAR100

# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

# Download the dataset
cifar100 = CIFAR100(root=os.path.expanduser("~/.cache"), download=True, train=False)

# Prepare the inputs
image, class_id = cifar100[3637]
image_input = preprocess(image).unsqueeze(0).to(device)
text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in cifar100.classes]).to(device)

# Calculate features
with torch.no_grad():
    image_features = model.encode_image(image_input)
    text_features = model.encode_text(text_inputs)

# Pick the top 5 most similar labels for the image
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
values, indices = similarity[0].topk(5)

# Print the result
print("\nTop predictions:\n")
for value, index in zip(values, indices):
    print(f"{cifar100.classes[index]:>16s}: {100 * value.item():.2f}%")

Files already downloaded and verified

Top predictions:

           snake: 65.43%
          turtle: 12.10%
    sweet_pepper: 3.87%
          lizard: 1.88%
       crocodile: 1.74%


In [97]:
import os
import clip
import torch

import numpy as np
from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR100
from tqdm import tqdm

# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

# Load the dataset
root = os.path.expanduser("~/.cache")
train = CIFAR100(root, download=True, train=True, transform=preprocess)
test = CIFAR100(root, download=True, train=False, transform=preprocess)



def get_features(dataset):
    all_features = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(DataLoader(dataset, batch_size=100)):
            features = model.encode_image(images.to(device))

            all_features.append(features)
            all_labels.append(labels)

    return torch.cat(all_features).cpu().numpy(), torch.cat(all_labels).cpu().numpy()

# Calculate the image features
train_features, train_labels = get_features(train)
test_features, test_labels = get_features(test)

# Perform logistic regression
classifier = LogisticRegression(random_state=0, C=0.316, max_iter=1000, verbose=1)
classifier.fit(train_features, train_labels)

# Evaluate using the logistic regression classifier
predictions = classifier.predict(test_features)
accuracy = np.mean((test_labels == predictions).astype(np.float)) * 100.
print(f"Accuracy = {accuracy:.3f}")

Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 100/100 [00:14<00:00,  6.71it/s]
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Accuracy = 79.980


/home/dlesser/anaconda3/envs/CLIP/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.4min finished


In [174]:
import torch.nn as nn
import torch.optim as optim
BATCH_SIZE=32
train_dataloader = CIFAR100(root, download=True, train=True, transform=preprocess) #Define your own dataloader
train = CIFAR100(root, download=True, train=True, transform=preprocess)
#https://github.com/openai/CLIP/issues/57
def convert_models_to_fp32(model): 
    for p in model.parameters(): 
        p.data = p.data.float() 
        p.grad.data = p.grad.data.float() 
        
def convert_models_to_mix(model):
    clip.model.convert_weights(model)

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32",device=device,jit=False) #Must set jit=False for training
clip.model.convert_weights(model)

loss_img = nn.KLDivLoss()
loss_txt = nn.KLDivLoss()
optimizer = optim.Adam(model.parameters(), lr=5e-5,betas=(0.9,0.98),eps=1e-6) #Params from paper


text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in cifar100.classes]).to(device)


for list_image,list_txt in tqdm(DataLoader(train, batch_size=BATCH_SIZE)):
    optimizer.zero_grad()

    #list_image,list_txt = batch #list_images is list of image in numpy array(np.uint8)
    #print(list_txt)
    #preprocess(Image.fromarray(img))
    #images= torch.stack([img for img in list_image],dim=0)
    texts = text_inputs[list_txt]
    texts = texts.to(device)
    list_image=list_image.to(device)
    
    #list_image=torch.unsqueeze(list_image,0).to(device)
    #texts = torch.unsqueeze(texts,0).to(device)
    
    logits_per_image, logits_per_text = model(list_image, texts)

    ground_truth = torch.arange(BATCH_SIZE).to(device)
    total_loss = (loss_img(logits_per_image,logits_per_text) + loss_txt(logits_per_text,logits_per_image,))/5e6
    #2e6 immediate improvement but decoheres next 2 iterations.
    #2e6
    total_loss.backward()
    
    #print('Total loss:',total_loss.item())

    convert_models_to_fp32(model)
    optimizer.step()
    clip.model.convert_weights(model)

Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 1563/1563 [04:35<00:00,  5.67it/s]


In [178]:
for i in range(25):
    for list_image,list_txt in tqdm(DataLoader(train, batch_size=BATCH_SIZE)):
        optimizer.zero_grad()

    #list_image,list_txt = batch #list_images is list of image in numpy array(np.uint8)
    #print(list_txt)
    #preprocess(Image.fromarray(img))
    #images= torch.stack([img for img in list_image],dim=0)
        texts = text_inputs[list_txt]
        texts = texts.to(device)
        list_image=list_image.to(device)
    
    #list_image=torch.unsqueeze(list_image,0).to(device)
    #texts = torch.unsqueeze(texts,0).to(device)
    
        logits_per_image, logits_per_text = model(list_image, texts)

    #ground_truth = torch.arange(BATCH_SIZE).to(device)
        total_loss = (loss_img(logits_per_image,logits_per_text) + loss_txt(logits_per_text,logits_per_image,))/5e6
        total_loss.backward()
    
    #print('Total loss:',total_loss.item())

        convert_models_to_fp32(model)
        optimizer.step()
        clip.model.convert_weights(model)

100%|██████████| 1563/1563 [04:26<00:00,  5.87it/s]


In [110]:
text_inputs[19]

tensor([49406,   320,  1125,   539,   320, 13644, 49407,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0], device='cuda:0')

In [179]:
import os
import clip
import torch
from torchvision.datasets import CIFAR100



# Download the dataset
cifar100 = CIFAR100(root=os.path.expanduser("~/.cache"), download=True, train=False)

# Prepare the inputs
image, class_id = cifar100[3637]
image_input = preprocess(image).unsqueeze(0).to(device)
text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in cifar100.classes]).to(device)

# Calculate features
with torch.no_grad():
    image_features = model.encode_image(image_input)
    text_features = model.encode_text(text_inputs)

# Pick the top 5 most similar labels for the image
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
values, indices = similarity[0].topk(5)

# Print the result
print("\nTop predictions:\n")
for value, index in zip(values, indices):
    print(f"{cifar100.classes[index]:>16s}: {100 * value.item():.2f}%")

Files already downloaded and verified

Top predictions:

           snake: 65.43%
          turtle: 12.10%
    sweet_pepper: 3.87%
          lizard: 1.88%
       crocodile: 1.74%


In [160]:
import os
import clip
import torch
from torchvision.datasets import CIFAR100



# Download the dataset
cifar100 = CIFAR100(root=os.path.expanduser("~/.cache"), download=True, train=False)

# Prepare the inputs
image, class_id = cifar100[3637]
image_input = preprocess(image).unsqueeze(0).to(device)
text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in cifar100.classes]).to(device)

# Calculate features
with torch.no_grad():
    image_features = model.encode_image(image_input)
    text_features = model.encode_text(text_inputs)

# Pick the top 5 most similar labels for the image
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
values, indices = similarity[0].topk(5)

# Print the result
print("\nTop predictions:\n")
for value, index in zip(values, indices):
    print(f"{cifar100.classes[index]:>16s}: {100 * value.item():.2f}%")

Files already downloaded and verified

Top predictions:

           snake: 70.80%
          turtle: 10.04%
          lizard: 1.86%
         leopard: 1.69%
       crocodile: 1.69%


In [169]:
import os
import clip
import torch

import numpy as np
from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR100
from tqdm import tqdm

# Load the model
#device = "cuda" if torch.cuda.is_available() else "cpu"
#model, preprocess = clip.load('ViT-B/32', device)

# Load the dataset
root = os.path.expanduser("~/.cache")
train = CIFAR100(root, download=True, train=True, transform=preprocess)
test = CIFAR100(root, download=True, train=False, transform=preprocess)



def get_features(dataset):
    all_features = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(DataLoader(dataset, batch_size=100)):
            features = model.encode_image(images.to(device))

            all_features.append(features)
            all_labels.append(labels)

    return torch.cat(all_features).cpu().numpy(), torch.cat(all_labels).cpu().numpy()

# Calculate the image features
train_features, train_labels = get_features(train)
test_features, test_labels = get_features(test)

# Perform logistic regression
classifier = LogisticRegression(random_state=0, C=0.316, max_iter=1000, verbose=1)
classifier.fit(train_features, train_labels)

# Evaluate using the logistic regression classifier
predictions = classifier.predict(test_features)
accuracy = np.mean((test_labels == predictions).astype(np.float)) * 100.
print(f"Accuracy = {accuracy:.3f}")

Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 100/100 [00:13<00:00,  7.28it/s]
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Accuracy = 33.220


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.4min finished


In [180]:
import os
import clip
import torch

import numpy as np
from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR100
from tqdm import tqdm

# Load the model
#device = "cuda" if torch.cuda.is_available() else "cpu"
#model, preprocess = clip.load('ViT-B/32', device)

# Load the dataset
root = os.path.expanduser("~/.cache")
train = CIFAR100(root, download=True, train=True, transform=preprocess)
test = CIFAR100(root, download=True, train=False, transform=preprocess)



def get_features(dataset):
    all_features = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(DataLoader(dataset, batch_size=100)):
            features = model.encode_image(images.to(device))

            all_features.append(features)
            all_labels.append(labels)

    return torch.cat(all_features).cpu().numpy(), torch.cat(all_labels).cpu().numpy()

# Calculate the image features
train_features, train_labels = get_features(train)
test_features, test_labels = get_features(test)

# Perform logistic regression
classifier = LogisticRegression(random_state=0, C=0.316, max_iter=1000, verbose=1)
classifier.fit(train_features, train_labels)

# Evaluate using the logistic regression classifier
predictions = classifier.predict(test_features)
accuracy = np.mean((test_labels == predictions).astype(np.float)) * 100.
print(f"Accuracy = {accuracy:.3f}")

Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 100/100 [00:14<00:00,  7.08it/s]
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Accuracy = 80.050


/home/dlesser/anaconda3/envs/CLIP/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.2min finished


In [161]:
import os
import clip
import torch

import numpy as np
from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR100
from tqdm import tqdm

# Load the model
#device = "cuda" if torch.cuda.is_available() else "cpu"
#model, preprocess = clip.load('ViT-B/32', device)

# Load the dataset
root = os.path.expanduser("~/.cache")
train = CIFAR100(root, download=True, train=True, transform=preprocess)
test = CIFAR100(root, download=True, train=False, transform=preprocess)



def get_features(dataset):
    all_features = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(DataLoader(dataset, batch_size=100)):
            features = model.encode_image(images.to(device))

            all_features.append(features)
            all_labels.append(labels)

    return torch.cat(all_features).cpu().numpy(), torch.cat(all_labels).cpu().numpy()

# Calculate the image features
train_features, train_labels = get_features(train)
test_features, test_labels = get_features(test)

# Perform logistic regression
#classifier = LogisticRegression(random_state=0, C=0.316, max_iter=1000, verbose=1)
#classifier.fit(train_features, train_labels)

# Evaluate using the logistic regression classifier
predictions = classifier.predict(test_features)
accuracy = np.mean((test_labels == predictions).astype(np.float)) * 100.
print(f"Accuracy = {accuracy:.3f}")

Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 100/100 [00:13<00:00,  7.32it/s]
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Accuracy = 80.230


/home/dlesser/anaconda3/envs/CLIP/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.2min finished


In [128]:
import os
import clip
import torch
from torchvision.datasets import CIFAR100

# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

# Download the dataset
cifar100 = CIFAR100(root=os.path.expanduser("~/.cache"), download=True, train=False)

# Prepare the inputs
image, class_id = cifar100[3637]
image_input = preprocess(image).unsqueeze(0).to(device)
text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in cifar100.classes]).to(device)

# Calculate features
with torch.no_grad():
    image_features = model.encode_image(image_input)
    text_features = model.encode_text(text_inputs)

# Pick the top 5 most similar labels for the image
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
values, indices = similarity[0].topk(5)

# Print the result
print("\nTop predictions:\n")
for value, index in zip(values, indices):
    print(f"{cifar100.classes[index]:>16s}: {100 * value.item():.2f}%")

Files already downloaded and verified

Top predictions:

           snake: 65.43%
          turtle: 12.10%
    sweet_pepper: 3.87%
          lizard: 1.88%
       crocodile: 1.74%


In [184]:
included=np.load('yfcc100m_subset_line_numbers.npy')

In [185]:
included[0]

2

In [188]:
included[14829395]

99999980